In [1]:
import pandas as pd
import numpy as np
import pickle
from itertools import chain
from collections import Counter
import operator

In [2]:
data_dir = '../prd_review_v2/data/'

prev_dt = '190104'
new_dt = '190117'

In [3]:
def get_df(dt):
    df_result = pd.read_pickle(data_dir+'df_result_'+dt+'.pkl')
    df_test = pd.read_pickle(data_dir+'df_test_'+dt+'.pkl')

    with open(data_dir+'word_index_'+dt+'.pkl', 'rb') as pkl_file:
        vocab_dict = pickle.load(pkl_file)            
    vocab_list = [x for x in range(len(vocab_dict)+1)]
    for k, v in vocab_dict.items():
        vocab_list[v] = k
        
    df_test = df_test.head(len(df_result)).reset_index(drop=True)
    # df_test.x = df_test.x.apply(lambda x: x.split())
    df_test = df_test.rename({'x': 'origin_tokens'}, axis=1)
    
    assert len(pd.Series(df_result.y == df_test.y).unique()) == 1
    
    df = pd.concat([df_result, df_test.drop(['y'], axis=1)], axis=1)
    df['tokens'] = df.x.apply(lambda x_list: [vocab_list[x] for x in x_list])
    
    return df

In [4]:
df_prev = get_df(prev_dt)
df_prev.sample(20)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
24564,"[50, 51, 85, 352, 13, 7, 1, 0, 0, 0, 0, 0, 0, ...",0,"[0.000114927236, 0.00014920074, 0.0001578473, ...",1,134600147,생각보다 많이 얇아요 코트안에 입기는 좋구요,"[생각, 많이, 얇, 코트, 안, 입, 좋]","[생각, 많이, 얇, 코트, 안, 입, 좋, 0, 0, 0, 0, 0, 0, 0, ..."
375,"[221, 93, 421, 34, 35, 36, 1263, 806, 356, 276...",1,"[6.070463e-06, 6.652022e-06, 4.094301e-06, 5.7...",1,135071197,최고에요 완전득템 넘 따뜻하고 예뻐요 은 꽉끼고 는 낙낙하게 신는 발볼 대박넓은 발...,"[최고, 완전, 득템, 넘, 따뜻, 예쁘, 250, 꽉, 끼, 255, 낙낙, 신,...","[최고, 완전, 득템, 넘, 따뜻, 예쁘, 250, 꽉, 끼, 255, 낙낙, 신,..."
38180,"[343, 26, 56, 408, 18, 6, 34, 1, 235, 0, 0, 0,...",1,"[1.847005e-05, 2.8906528e-05, 3.0397416e-05, 3...",1,134836641,매우 만족합니다 제가 매번쓰보지만 넘 좋아요 강추입니다,"[매우, 만족, 제, 매번, 쓰, 보, 넘, 좋, 강추]","[매우, 만족, 제, 매번, 쓰, 보, 넘, 좋, 강추, 0, 0, 0, 0, 0,..."
11790,"[128, 12, 1917, 179, 1049, 142, 74, 584, 12, 1...",1,"[4.6345504e-05, 2.5787334e-05, 3.553094e-05, 3...",1,135106139,항상 사용하는 비비크림이예요 번들거림없이 피부에 밀착되어 사용감좋아요,"[항상, 사용, 비비, 크림, 번들거리, 없이, 피부, 밀착, 사용, 좋]","[항상, 사용, 비비, 크림, 번들거리, 없이, 피부, 밀착, 사용, 좋, 0, 0..."
29783,"[128, 12, 19, 903, 70, 24, 6, 42, 1, 0, 0, 0, ...",1,"[1.5564548e-05, 8.660349e-06, 1.7795914e-05, 8...",1,135010190,항상 사용하는제품인데 새로운 색으로 사봤어요 이뻤으면좋겠네요,"[항상, 사용, 제품, 새롭, 색, 사, 보, 이쁘, 좋]","[항상, 사용, 제품, 새롭, 색, 사, 보, 이쁘, 좋, 0, 0, 0, 0, 0..."
37580,"[8, 407, 352, 1, 409, 1481, 138, 3, 8, 46, 109...",0,"[0.00015302445, 0.00033447848, 0.00017401419, ...",0,134634105,같은 브랜드 코트는 좋은데 터들넥은 별로인것 같아요 옷감은 가벼운 편이긴 하나 금방...,"[같, 브랜드, 코트, 좋, 터, 넥, 별로, 것, 같, 옷, 감, 가볍, 편, 하...","[같, 브랜드, 코트, 좋, 터, 넥, 별로, 것, 같, 옷, 감, 가볍, 편, 하..."
12039,"[825, 106, 216, 4, 5, 126, 63, 64, 3, 8, 0, 0,...",1,"[5.2197196e-05, 3.788488e-05, 4.4277007e-05, 4...",1,135423702,어느 정도 효과가 있는지는 잘 모르겠지만 느낌은 괜찮은 것 같네요,"[어느, 정도, 효과, 있, 잘, 모르, 느낌, 괜찮, 것, 같]","[어느, 정도, 효과, 있, 잘, 모르, 느낌, 괜찮, 것, 같, 0, 0, 0, ..."
27033,"[2036, 33, 366, 473, 32, 1, 1435, 1517, 947, 1...",1,"[6.0867656e-06, 1.67938e-06, 2.1408412e-06, 4....",1,134932796,다섯번째 울부인데 편하고 좋아요 아디다스 세일기간에 카드 쿠폰 할인 적용하니까 만원...,"[다섯, 번, 울, 부, 편하, 좋, 아디, 다스, 세일, 기간, 카드, 쿠폰, 할...","[다섯, 번, 울, 부, 편하, 좋, 아디, 다스, 세일, 기간, 카드, 쿠폰, 할..."
39386,"[79, 2, 20, 277, 414, 79, 3, 268, 26, 0, 0, 0,...",1,"[9.265052e-05, 0.00011192992, 9.119137e-05, 8....",1,135133942,촉촉하긴한데 커버력이 약함 촉촉한거 찾던차라 만족입니다,"[촉촉, 하, 크, 버력, 약하, 촉촉, 것, 찾, 차라, 만족]","[촉촉, 하, 크, 버력, 약하, 촉촉, 것, 찾, 만족, 0, 0, 0, 0, 0..."
31222,"[180, 331, 165, 52, 5, 24, 54, 42, 2, 0, 0, 0,...",1,"[1.692746e-05, 1.9289337e-05, 1.1629471e-05, 3...",1,134889657,원하듼 스타일의신발 저렴하게 잘샀어요 다들 이쁘다고 하네요,"[원, 스타일, 신발, 저렴, 잘, 사, 다, 이쁘, 하]","[원, 스타일, 신발, 저렴, 잘, 사, 다, 이쁘, 하, 0, 0, 0, 0, 0..."


In [5]:
df_new = get_df(new_dt)
df_new.sample(20)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
26853,"[1, 53, 686, 46, 13, 1, 534, 1026, 7, 1630, 0,...",1,"[7.602181e-05, 0.00017351305, 9.7230135e-05, 0...",1,134642558,좋은상품 비교하고 저렴하게 구매해서 좋네요 단 구김이 너무심해요,"[좋, 상품, 비교, 저렴, 구매, 좋, 단, 구김, 너무, 심해요]","[좋, 상품, 비교, 저렴, 구매, 좋, 단, 구김, 너무, 심해요, 0, 0, 0..."
25095,"[9, 843, 31, 11, 94, 1, 0, 0, 0, 0, 0, 0, 0, 0...",1,"[0.00056321966, 0.002523439, 0.002966772, 0.00...",1,134605742,사용전이라 써봐야 알듯 좋겠죠,"[사용, 전이, 쓰, 보, 알, 좋]","[사용, 전이, 쓰, 보, 알, 좋, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
46833,"[1135, 2417, 199, 506, 5, 107, 3, 14, 1182, 14...",0,"[9.392091e-05, 0.00030655463, 0.00016913576, 0...",0,134714633,강아지가 빗질하기 싫어하네요 털이 잘 안풀려요 단모용인다봅니다,"[강아지, 빗질, 질, 싫, 하, 털, 잘, 안, 풀리, 단모, 용, 일, 봅니다]","[강아지, 빗질, 질, 싫, 하, 털, 잘, 안, 풀리, 용, 일, 봅니다, 0, ..."
51509,"[2409, 38, 15, 1654, 7, 1, 365, 50, 19, 187, 4...",0,"[8.720125e-05, 5.152357e-05, 0.00041708886, 0....",1,135842965,다자인도 예쁘고 가격도 착해서 너무좋아요 그런데 옷이 좀 무겁고 배부분이 사이즈가 ...,"[다자인, 예쁘, 가격, 착해서, 너무, 좋, 그런데, 옷, 좀, 무겁, 배, 부분...","[다자인, 예쁘, 가격, 착해서, 너무, 좋, 그런데, 옷, 좀, 무겁, 배, 부분..."
47086,"[397, 1023, 15, 480, 43, 536, 249, 1660, 32, 1...",0,"[0.0017857504, 0.0026676932, 0.011637483, 0.00...",0,134697963,타 업체랑 가격차이 많이 심하게 남 알아보고 사세요 배송도 완전 느림,"[타, 업체, 가격, 차이, 많이, 심하, 남, 알아보, 살, 배송, 완전, 느림]","[타, 업체, 가격, 차이, 많이, 심하, 남, 알아보, 살, 배송, 완전, 느림,..."
33200,"[46, 160, 13, 125, 157, 4, 2, 1, 0, 0, 0, 0, 0...",1,"[0.0037893723, 0.0023318678, 0.00060184556, 0....",1,135138944,저렴하게 넉넉하게 구매하니 항상 깔끔하게 입을수있어 좋아요,"[저렴, 넉넉, 구매, 항상, 깔끔, 입, 있, 좋]","[저렴, 넉넉, 구매, 항상, 깔끔, 입, 있, 좋, 0, 0, 0, 0, 0, 0..."
34238,"[287, 287, 1, 5, 161, 1638, 6148, 6, 1, 76, 40...",1,"[0.0015264036, 0.0016342751, 0.000767165, 0.00...",1,134994094,핑크핑크한거 좋아하는 딸램이지만 아가씨 같다니 좋아합니다 이뻐요,"[핑크, 핑크, 좋, 하, 딸, 램, 아가씨, 같, 좋, 합니다, 이쁘]","[핑크, 핑크, 좋, 하, 딸, 램, 아가씨, 같, 좋, 합니다, 이쁘, 0, 0,..."
17972,"[126, 10, 316, 118, 20, 86, 1205, 1317, 234, 7...",1,"[6.1819526e-05, 2.7383412e-05, 2.9648452e-05, ...",1,134819873,믿고쓰는 견미리팩트 없으면 완전 불안해요 광채도 커버도 너무 자연스럽고 예쁘게 피부...,"[믿, 쓸, 견미리, 팩트, 없, 완전, 불안, 광채, 커버, 너무, 자연, 예쁘,...","[믿, 쓸, 견미리, 팩트, 없, 완전, 불안, 광채, 커버, 너무, 자연, 예쁘,..."
40182,"[139, 139, 139, 66, 446, 47, 135, 1, 13, 1791,...",1,"[0.00022286519, 6.076657e-05, 2.5941616e-05, 3...",1,135642666,굿굿굿 가볍고 너무너무 편하네요 상품평이 좋아서 구매했는데 최곱니다,"[굿, 굿, 굿, 가볍, 너무너무, 편하, 상품평, 좋, 구매, 곱]","[굿, 굿, 굿, 가볍, 너무너무, 편하, 상품평, 좋, 구매, 곱, 0, 0, 0..."
11766,"[4664, 2782, 18, 13, 100, 89, 1676, 227, 516, ...",1,"[0.0007865573, 0.00035956758, 0.00019019464, 6...",1,135410265,친환경제품이라 구매했어요 처음 향이 강한듯했지만 시간지나니 은은하니 끈적임 없어 좋...,"[친환경, 환경, 제품, 구매, 처음, 향, 강한, 시간, 지나, 은은, 끈적임, ...","[친환경, 환경, 제품, 구매, 처음, 향, 강한, 시간, 지나, 은은, 끈적임, ..."


In [6]:
# attention model 내에서 alpha값이 큰 순서대로 index 정렬
# "UNKNOWN" seq == 0 최초로 "UNKNOWN"이후 토큰들은 모두 "UNKNOWN"이므로 고려하지 않음
# (고려되는 토큰 길이 / 3) + 1개 고려
def get_max_args(row):
    alpha = np.array(row['alpha'])
    x =  np.array(row['x'])
    zero_index = np.where(x == 0)[0].tolist()
    if zero_index:
        sent_size = int(zero_index[0])
    else:
        sent_size = int(len(x))
    num_alpha = int(sent_size / 3) + 1
    alpha = alpha[:sent_size].argsort()[-num_alpha:][::-1]
    return alpha.tolist()

In [7]:
df_prev['max_args'] = df_prev.apply(lambda row: get_max_args(row), axis=1)
df_new['max_args'] = df_new.apply(lambda row: get_max_args(row), axis=1)

In [8]:
# 위에서 산출한 token idx를 token으로변환
def get_alpha_tokens(row):
    max_args = row['max_args']
    xs = row['x']
    tokens = row['tokens']
    alpha_tokens = list()
    try:
        alpha_tokens = [tokens[arg] for arg in max_args if xs[arg] != 0]
    except IndexError:
        print("IndexError")
        alpha_tokens = None
    finally:
        return alpha_tokens

In [9]:
df_prev['alpha_tokens'] = df_prev.apply(lambda row: get_alpha_tokens(row), axis=1)
df_new['alpha_tokens'] = df_new.apply(lambda row: get_alpha_tokens(row), axis=1)

In [11]:
df_new.loc[:, ['text', 'alpha_tokens', 'tokens', 'y', 'pred']].sample(20)

,text,alpha_tokens,tokens,y,pred
40098,배송도빠르고아직써보진않았지만기대합니다,"[배송, 아직, 빠르]","[배송, 빠르, 아직, 쓰, 보, 않, 기대, 0, 0, 0, 0, 0, 0, 0,...",0,1
44624,아직 써보진 안아서 효과있기를 바랍니다,"[효과, 바랍니다, 있]","[아직, 쓰, 보, 알, 효과, 있, 바랍니다, 0, 0, 0, 0, 0, 0, 0...",1,1
46272,사진으로 볼때는 나무재질이 조금 좋아 보였는데 재질이 까칠까칠 영 안좋습니다 딱 그...,"[입, 가격, 안, 그, 좋, 영]","[사진, 볼, 때, 나무, 재질, 조금, 좋, 보이, 재질, 영, 안, 좋, 딱, ...",0,0
44308,앉았다 일어섰다 아주 편하고 스타일 죽입니다,"[죽, 앉]","[앉, 아주, 편하, 스타일, 죽, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,1
13808,좋긴한데 LED가 맞나요 거의 LCD 느낌이고 엄청 두껍고 무겁네요,"[무겁, 느낌, led, 거의]","[좋, 한데, led, 맞, 거의, 느낌, 엄청, 두껍, 무겁, 0, 0, 0, 0...",0,0
16279,디따얇아요 어차피 빨리갈아주려고 저렴하고 얇은걸찾았지만 더 저렴한것도있었기에 이건 ...,"[얇, 얇, 좋, 어차피, 저렴, 찾, 빨리]","[얇, 어차피, 빨리, 갈아주, 저렴, 얇, 찾, 더, 저렴, 있, 이건, 비싼, ...",0,1
46748,할인받아서잘산듯ㅋ남편이좋아해용ㅋ,"[해용, ㅋ, 좋, ㅋㅋ]","[할인, 받, 잘, 살, ㅋㅋ, 남편, 좋, 해용, ㅋ, 0, 0, 0, 0, 0,...",1,1
426,아이가 너무 좋아했는데 몇일만에 잃어버려서 다시 구마합니다 디자인 가격 모두 만족합니다,"[가격, 만족, 모두, 일, 잃어버리]","[아이, 너무, 좋, 했, 몇, 일, 잃어버리, 다시, 구마, 디자인, 가격, 모두...",1,1
14062,너무 기대했던 제품이라 역시 좋네요 그릇도 너무 예뻐요,"[예쁘, 그릇, 너무]","[너무, 기대, 제품, 역시, 좋, 그릇, 너무, 예쁘, 0, 0, 0, 0, 0,...",1,1
40662,지난 구매에서도 잘 사용했는데 이번에 또 구매했어요 앞으로도 계속사용할 생각입니다 ...,"[같, 낫, 젤, 판매, 쇼, 생각, 홈]","[지난, 구매, 잘, 사용, 이번, 번, 또, 구매, 앞, 계속, 사용, 생각, 홈...",1,1


In [12]:
df_new.loc[(df_new.y == 0) & (df_new.pred == 0), ['text', 'alpha_tokens', 'tokens']].sample(20)

,text,alpha_tokens,tokens
25387,배송은 빨랐어요 생각보다 품질이 별로에요 너무기대했나봄,"[별로, 봄, 기대]","[배송, 빨랐, 생각, 품질, 별로, 너무, 기대, 봄, 0, 0, 0, 0, 0,..."
28109,이건뭔가요 지퍼가 고장나서 왔어요 이쪽으로 닫아도 열리고 반대쪽으로해도 열리는 ㅠ귀...,"[안, 반품, 합니다, 해도, 열리]","[이건, 뭔가요, 지퍼, 고장나, 왔, 닫, 열리, 반대, 해도, 열리, 반품, 안..."
41664,이건 아닌거 같아요 이러면 뽑아쓰는 이유가 없잖아요,"[없, 이유, 뽑]","[이건, 같, 이러, 뽑, 쓸, 이유, 없, 않, 0, 0, 0, 0, 0, 0, ..."
15517,뒤축이 구겨져 왔어요 박음질도 좀 허술하고 오래된 재고품을 받은 느낌입니다 전에 다...,"[참, 벌, 상품, 맘, 들, 샀, 복불복, 번, 모델, 다른, 이번]","[뒤축, 축, 구기, 지, 왔, 박음질, 좀, 허술, 오래, 된, 재고품, 품, 받..."
10711,내구도 최악이네요 붙인지 일만에 금가버리네요 어째 이런걸 파는지,"[팔, 이런, 어찌, 버리]","[내구, 최악, 붙인, 3, 일, 버리, 어찌, 이런, 팔, 0, 0, 0, 0, ..."
44153,왕짜증나네 물건붙이지도않고 배송접수시켜서 취소못하게하더니 왜화요일배송할물건을 월요일...,"[안, 하, 이해, 되, 물건, 리, 못, 어디, 도대체, 배송, 완료]","[왕, 짜증, 날, 물건, 붙이, 않, 배송, 접수, 취소, 못, 하, 하, 왜, ..."
3262,상품평보고 구매했는데 솔직히 뭐가좋은건지 전혀모르겠어요 향은강해요 근데 부드러워진다...,"[바랍니다, 그러, 모르, 쓸, 보, 나, 지]","[상품평, 구매, 솔직히, 뭣, 좋, 전혀, 모르, 향, 강해요, 근데, 부드럽, ..."
48389,저렴해서 구입했는데 별로네요 집에서 입혀야겠어요,"[하, 별로, 입히]","[저렴, 구입, 별로, 집, 입히, 하, 0, 0, 0, 0, 0, 0, 0, 0,..."
27379,세탁 하려고 울샴푸에 담근 동시에 초록 물감 풀어놓은 것처럼 엄청 물 빠지네요 헹궈...,"[나와요, 초록색, 계속, 색, 빠지]","[세탁, 하, 울, 샴푸, 동시, 초록, 물감, 엄청, 물, 빠지, 계속, 초록색,..."
7687,아무리저렴한상품이라도검수는해서보내야하는거아닌가요 종이테이프가붙어있네요 떼어내면하얗게...,"[안, 신청, 하, 반품, 합니다, 너무, 싼, 같]","[아무리, 저렴, 상품, 검수, 해서, 보내, 하, 테이프, 붙, 있, 떼, 내, ..."


In [13]:
df_new.loc[(df_new.y == 1) & (df_new.pred == 1), ['text', 'alpha_tokens', 'tokens']].sample(20)

,text,alpha_tokens,tokens
41164,넉넉한 크기로 신청해서 품도 여유있고 까슬하고 탄력도 좋아 만족합니다,"[만족, 넉넉, 신청, 크기]","[넉넉, 크기, 신청, 품, 여유, 있, 탄력, 좋, 만족, 0, 0, 0, 0, ..."
34082,색깔이 기존 의자랑 너무 잘 맞아요 커버를 씌운 것이 아니라 원래 의자 같아요 조금...,"[만족, 큰, 조금, 커버, 색깔]","[색깔, 기존, 의자, 너무, 잘, 맞, 커버, 원래, 의자, 같, 조금, 커버, ..."
7373,잘쓰고 있어요 요즘 매번 미게먼지 나쁨이라던데 나갈때 하나씩 가지고 다녀요 감사합니다♡,"[밀, 잘, 먼지, 매번, 쓸, 감사]","[잘, 쓸, 있, 요즘, 매번, 밀, 먼지, 나쁨, 나갈, 때, 하나, 씩, 가지,..."
32531,노견울애기 이거는 참잘먹네요,"[잘, 먹]","[애기, 이거, 참, 잘, 먹, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
14728,오자마자 남편이 편하고 예쁘다며 신고 모임에 나갔어요 저렴하게 잘사서 좋아요,"[좋, 잘살, 저렴, 나갔]","[오, 남편, 편하, 예쁘, 신고, 모임, 나갔, 저렴, 잘살, 좋, 0, 0, 0..."
38370,샴푸를 사용하고 있어서 고민없이 주문했는데 역시나 너무 좋아요 향수를 뿌리지 않아도...,"[샴푸, 네, 냄새, 나, 고민, 사용, 좋]","[샴푸, 사용, 있, 고민, 없이, 주문, 역시, 나, 너무, 좋, 향수, 뿌리, ..."
47476,저렴하게 잘 구입했어요 이쁘게 잘 쓰고 있답니다,"[쓸, 있, 잘]","[저렴, 잘, 구입, 이쁘, 잘, 쓸, 있, 0, 0, 0, 0, 0, 0, 0, ..."
12422,기 에 키로 반 주로입는데 홈쇼핑으로 바지는 거의 실패 근데 이건 디자인 톡톡한재질...,"[제, 샀, 근데, 작, 재질, 나왔, 개, 디자인, 다, 이건, 톡톡]","[기, 164, 54, 55, 반, 주로, 입, 홈, 쇼핑, 바지, 실패, 근데, ..."
17302,사이즈 넉넉합니다 기장도 길어요 제가 넘는데 무릎 밑에까지 옵니다 맘에 들어요,"[맘, 들, 175, 기장, 제]","[사이즈, 넉넉, 기장, 길, 제, 175, 넘, 무릎, 밑, 옵니다, 맘, 들, ..."
32959,한번썼는데 촉촉하고 얼굴에 윤이나니 넘 좋아요 기초끝내고 뿌려주고 빠데 하고 또 뿌...,"[쓸, 잘, 오래가, 정말, 줬, 뿌리, 또]","[한, 번, 쓰, 촉촉, 얼굴, 윤, 넘, 좋, 기초, 끝내, 뿌리, 주, 하, 또..."


In [15]:
df_prev['correct'] = (df_prev.y == df_prev.pred)
df_new['correct'] = (df_new.y == df_new.pred)

In [16]:
def get_accuarcy(df):
    return len(df.loc[(df.correct == True)]) / len(df)


def get_f1_score(df):
    tp = len(df.loc[(df.y == 1) & (df.pred == 1)])
    fp = len(df.loc[(df.y == 0) & (df.pred == 1)])
    tn = len(df.loc[(df.y == 1) & (df.pred == 0)])
    fn = len(df.loc[(df.y == 0) & (df.pred == 0)])
    
    pr_0 = fn / (fp+fn)
    rc_0 = fn / (tn+fn)
    f1_0 = 2*pr_0*rc_0/(pr_0+rc_0)

    pr_1 = tp / (tp+tn)
    rc_1 = tp / (tp+fp)
    f1_1 = 2*pr_1*rc_1/(pr_1+rc_1)

    return f1_0, f1_1

In [17]:
acc_prev = get_accuarcy(df_prev)
acc_new = get_accuarcy(df_new)
print("acc_prev : {}, acc_new: {}".format(acc_prev, acc_new))

acc_prev : 0.852729301948052, acc_new: 0.860129176980198


In [18]:
f1_neg_prev, f1_pos_prev = get_f1_score(df_prev)
f1_neg_new, f1_pos_new = get_f1_score(df_new)
print("f1_neg_prev : {}, f1_neg_new: {}".format(f1_neg_prev, f1_neg_new))
print("f1_pos_prev : {}, f1_pos_new: {}".format(f1_pos_prev, f1_pos_new))

f1_neg_prev : 0.6666666666666667, f1_neg_new: 0.6945136630485282
f1_pos_prev : 0.9054859189321179, f1_pos_new: 0.9093006633478375


In [19]:
alpha_tokens = df_new.loc[(df_new.correct == True) & (df_new.y == 1)].alpha_tokens.tolist()
counter = Counter(chain(*alpha_tokens))
alpha_count = sorted(counter.items(), key=operator.itemgetter(1))
alpha_count.reverse()